In [1]:
%%writefile d:\myPyCode\팀프로젝트\netflix_Unofficial_API_module.py

import requests
import pandas as pd
# from bs4 import BeautifulSoup
# import urllib.request
import json

url = "https://netflix-unofficial.p.rapidapi.com/api/"


select_netflix_mode = ['genres','search','countries']
# genres,countries = 항목별 뽑아야한다.
# search = 분류로 나눠야 한다.


headers = {
    'x-rapidapi-key': "c05cbb963bmshc6a47b50ae8c81cp167c7bjsn5f651e0b6361",
    'x-rapidapi-host': "netflix-unofficial.p.rapidapi.com"
    }

netflix_data = requests.request("GET", url+select_netflix_mode[1], headers=headers).json()

print(netflix_data)

def netflix_init():
    netflix_keys_list = ['title']
    
    for a in json.loads(netflix_data[""]).keys() :
        netflix_keys_list.append(a)
        
    f = open("netflix_unofficial.csv","w",-1,"utf-8")
    for i in netflix_keys_list : 
        f.write(i+',')
    f.write('\n')

    for data_title in netflix_data.keys() :
        f.write(data_title.replace(','," ")+',')
        data_get_key = json.loads(netflix_data[data_title])
        for get_key in netflix_keys_list[1:] :
            f.write(str(data_get_key[get_key]).replace(',',' ')+',')

            # print(data_get_key[get_key])
        f.write('\n')

    f.close()





Overwriting d:\myPyCode\팀프로젝트\netflix_Unofficial_API_module.py


In [5]:
%%writefile d:\myPyCode\팀프로젝트\netflix_frame_module.py

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import csv
import seaborn as sns
import json

df = pd.read_csv("netflix_titles.csv",encoding="latin1")
select = pd.DataFrame(df,columns = ['show_id', 'type', 'title','country', 'date_added','release_year', 'rating',
'duration','listed_in', 'description'])

# 빈 값 처리
select = select.fillna(method='ffill')

# 초기 데이터프레임 뽑아내기
def netflix_data_init() :
    df = pd.read_csv("netflix_titles.csv",encoding="latin1")
    select = pd.DataFrame(df,columns = ['show_id', 'type', 'title','country', 'date_added','release_year',
    'rating','duration','listed_in', 'description'])
    return select

# 날짜별순으로 값 원하는 값 뽑아내기 (type과 key값 )
def netflix_date_selection(select_col,Name) :
    #select.sort_values(by = ['release_year','date_added']) 날짜별로 구분하기
    selection = select[select[select_col] == Name].sort_values(by = ['release_year','date_added'])
    #select[select[select_col] == Name] Name과 같은 select key값을 뽑아낸다.
    return selection

# 날짜순으로 타입안에 원하는 문자포함되어 있는 행 뽑아내기
def netflix_data_findword(netflix_framedata,position,Name) :
    find_line = netflix_framedata.loc[netflix_framedata[position].str.contains(Name)].sort_values(by= 
    ['release_year','date_added'])
    return find_line

# 중복 없이 타입 뽑아내기
def remove_overlap(netflix_data,netflix_want_name):
    list_module = []
    netflix_data_find = set(list(netflix_data[netflix_want_name]))
    for i in netflix_data_find :
        list_module.append(i)
    return list_module

# 데이터프레임에서 반복 갯수세기
def netflix_count(netflix_dataframe,collect_type) :
    collect_lists = netflix_dataframe[collect_type].value_counts()
    return collect_lists
    
# 원 그래프 만들기
# def circle_grape(label_X,label_Y):
#     plt.pie(label_X)
#     plt.pie(label_X, labels=label_Y, autopct='%0.1f%%')
#     plt.show()


# 선형그래프 뽑아내기
def analyze_netflix_year(data_X,data_Y,data_title):
    p1 = plt.plot(data_X,data_Y,'r^--',label=data_title)
    plt.legend()
    plt.show()

# 산점도 그래프 그리기
def circle_3D_grape(X,Y,Z):
    ax = plt.figure().add_subplot(111,projection='3d')
    ax.scatter(X,Y,Z)

    ax.set_xlabel('year')
    ax.set_ylabel('type')
    ax.set_zlabel('age')
    plt.suptitle('aaaa',fontsize=16)
    plt.show()


# 데이터 정렬하기 1) 영화장르
def netflix_data_genre_years_clear(movie_listed_in,years_listed_in, netflix_data):

    # 시대별 영화장르 뽑아내기
    df = pd.DataFrame( )
    df = df.reindex(index = years_listed_in)

    for movie_elements in movie_listed_in :
    
        #원하는장르포함 모든 행 뽑기
        listed_in_pullout = netflix_data_findword(netflix_data,'listed_in', movie_elements)
        # 시대별 기준으로 나누기
        listed_in_count = netflix_count(listed_in_pullout,'release_year') 
        # 아이템으로 정렬하면 key값을 정렬가능
        listed_in_count = dict(sorted(listed_in_count.items(),reverse=True))
        # dataframe 초기 설정
        df = df.reindex(index=years_listed_in)
        df[movie_elements] = 0
    
        for index_elements in years_listed_in :
            if index_elements in list(listed_in_count.keys()) :
                df.loc[index_elements,[movie_elements]]=listed_in_count[index_elements]

    return df

# 데이터 정렬하기 1) 나이 장르
def netflix_data_rating_years_clear(age_listed_in,years_listed_in, netflix_data):

    # 시대별 영화장르 뽑아내기
    df = pd.DataFrame( )
    df = df.reindex(index = years_listed_in)

    for age_elements in age_listed_in :
    
        #원하는장르포함 모든 행 뽑기
        listed_in_pullout = netflix_data_findword(netflix_data,'rating', age_elements)
        # 시대별 기준으로 나누기
        listed_in_count = netflix_count(listed_in_pullout,'release_year') 
        # 아이템으로 정렬하면 key값을 정렬가능
        listed_in_count = dict(sorted(listed_in_count.items(),reverse=True))
        # dataframe 초기 설정
        df = df.reindex(index=years_listed_in)
        df[age_elements] = 0
    
        for index_elements in years_listed_in :
            if index_elements in list(listed_in_count.keys()) :
                df.loc[index_elements,[age_elements]]=listed_in_count[index_elements]

    return df

# 나라를 기준으로 데이터 추출
def grape_step1(netflix_data,movie_listed_in,years_country_listed_in,country_listed_in):
    a=1
    fig = plt.figure(figsize=(9,9)) 
    # 나라별 데이터 뽑기
    for netflix_country_lists in country_listed_in :
        netflix_country_data =  netflix_data_findword(netflix_data,'country',netflix_country_lists)
        netflix_country_dataFrame = netflix_data_genre_years_clear(movie_listed_in,years_country_listed_in, 
        netflix_country_data)
        # 막대그래프화
        netflix_country_dataFrame.plot(kind='bar',title= netflix_country_lists,ax=fig.add_subplot(1,3,a))
        a=a+1
        plt.show()

def grape_step3(netflix_data,years_age_listed_in,country_listed_in):
    a=1
    fig = plt.figure(figsize=(9,9)) 
    # 나이별 데이터 뽑기
    for netflix_country_lists in country_listed_in :
        netflix_country_data =  netflix_data_findword(netflix_data,'country',netflix_country_lists)
        # 연령 시청 종류 조사
        netflix_age_list = remove_overlap(netflix_country_data,'rating')
        # 연령별 데이터 뽑기
        netflix_age_dataFrame = netflix_data_rating_years_clear(netflix_age_list, years_age_listed_in,
        netflix_country_data)
        # 막대그래프화
        netflix_age_dataFrame.plot(kind='bar', title= netflix_country_lists, ax=fig.add_subplot(1,3,a))
        a=a+1
        plt.show()

# step 4 나라 하나 당 연도별 그래프 뽑기 
# 그래프 속 내용 : 장르별, 나이별 :  count

def grape_step4_pre(year,nation,netflix_age,movie_listed_in):
    # 년도별 뽑는다.
    select_years = netflix_date_selection('release_year',year)
    # 년도별로 뽑은걸 나라별로 뽑는다.
    select_year = select_years[select_years['country'] == nation].sort_values(by = ['release_year','date_added'])
    
    # 비어있는 데이터프레임 생성
    df = pd.DataFrame()
    # 행값의 네임 설정 (netflix_age)
    df = df.reindex(index=netflix_age)
    
    #데이터 프레임 설정 장르별,나이별 count
    for a in netflix_age :
        find_age_year = netflix_data_findword(select_year,'rating',a)
        for i in movie_listed_in :
            select_movie_year = netflix_data_findword(find_age_year,'listed_in',i)
            select_age_movie_year = dict(netflix_count(select_movie_year,'listed_in'))
            df.loc[a,[i]]=len(select_age_movie_year)

    sns.heatmap(df,annot=True,fmt='0.0f',annot_kws={"size": 5})
    plt.title(str(year)+' - '+nation,fontsize=5)
    plt.show()
#https://analytics4everything.tistory.com/66 참고 

def grape_step4(country_listed_in,movie_listed_in):
    netflix_age = remove_overlap(select,'rating')
    a=1
    sns.set(font_scale=0.5)
    fig = plt.figure(figsize=(9,9))
    
    for country_list in country_listed_in:
        for year_num in range(2017,2020):
            fig.add_subplot(3,3,a)
            grape_step4_pre(year_num,country_list,netflix_age,movie_listed_in)
            a=a+1

# 행 열 변환 팁
# listed_in_count.keys()
# a = pd.DataFrame(comedy_count,index=['장르'])
# a = a.transpose()
# a

    



Overwriting d:\myPyCode\팀프로젝트\netflix_frame_module.py


In [1]:
cd d:\myPyCode\팀프로젝트

d:\myPyCode\팀프로젝트


In [2]:
%matplotlib tk

import pandas as pd
import numpy as np
# from matplotlib import pyplot as plt
from netflix_frame_module import *
import seaborn as sns
# import json
# from netflix_Unofficial_API_module import netflix_init

movie_listed_in = ['Comed','Horror','Independ','Sci',
                   'Thrille','Drama','Classic','Adventure',
                  'Family','Romantic','Music','Classic',
                  'International']

years_country_listed_in = list(range(2010,2020))
years_age_listed_in = list(range(2016,2021))
country_listed_in = ['United States','South Korea','United Kingdom']
# 초기 데이터
netflix_data = netflix_data_init()
# 데이터 프레임 null 값 처리하기
netflix_data = netflix_data.fillna(method='ffill')
grape_step1(netflix_data,movie_listed_in,years_country_listed_in,country_listed_in)
grape_step3(netflix_data,years_age_listed_in,country_listed_in)
grape_step4(country_listed_in,movie_listed_in)



In [ ]:
# comedy_pullout = netflix_data_findword('listed_in','Comed')
# comedy_count = netflix_count(comedy_pullout,'release_year')
# comedy_lists = netflix_count(comedy_pullout,'release_year').keys()
# comedy_count = dict(sorted(comedy_count.items(),reverse=True))
# analyze_netflix_year(list(comedy_count.keys()),list(comedy_count.values()),'comedy')
# analyze_netflix_year


In [1]:
cd d:\myPyCode\팀프로젝트

c:\myPyCode\팀프로젝트


In [11]:

%matplotlib tk

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from netflix_frame_module import *
from netflix_Unofficial_API_module import netflix_init

comedy_pullout = netflix_data_findword('listed_in','Comed')
comedy_count = netflix_count(comedy_pullout,'release_year')
comedy_lists = netflix_count(comedy_pullout,'release_year').keys()
comedy_count = dict(sorted(comedy_count.items(),reverse=True))



,장르
2020,9
2019,309
2018,337
2017,300
2016,227
2015,150
2014,90
2013,72
2012,78
2011,51


In [ ]:
a = pd.DataFrame(comedy_count,index=['장르'])
a = a.transpose()
a
# a.columns.names=['연도별']
plt.subplots(1,1)
a.plot(kind='bar')
plt.show()


In [29]:
list_module = []

a = netflix_data_findword(netflix_data,'type','Movie')
b = set(list(a['listed_in']))
for i in b :
    list_module.append(i)
list_module


['Children & Family Movies',
 'Action & Adventure, Cult Movies, Dramas',
 'Children & Family Movies, Comedies, Dramas',
 'Action & Adventure, Anime Features, Children & Family Movies',
 'Action & Adventure, Comedies, Romantic Movies',
 'Dramas, Faith & Spirituality, Romantic Movies',
 'Comedies, LGBTQ Movies',
 'Classic Movies, Documentaries',
 'Action & Adventure, Comedies, Horror Movies',
 'Classic Movies, Dramas',
 'Dramas, LGBTQ Movies',
 'Children & Family Movies, Dramas',
 'Comedies, Documentaries',
 'Comedies, Faith & Spirituality, Romantic Movies',
 'Comedies, Dramas, LGBTQ Movies',
 'Sports Movies',
 'Children & Family Movies, Dramas, Sports Movies',
 'Classic Movies, Dramas, LGBTQ Movies',
 'Anime Features, International Movies, Romantic Movies',
 'Comedies, Cult Movies, Sports Movies',
 'Children & Family Movies, Documentaries, International Movies',
 'Children & Family Movies, Dramas, Romantic Movies',
 'Sci-Fi & Fantasy',
 'Classic Movies, Comedies, Dramas',
 'Children & F

In [ ]:

# list_module = []
# a = netflix_data_findword(netflix_data,'listed_in','Comed')
# b = set(list(a['rating']))
# for i in b :
#     list_module.append(i)
# list_module